In [1]:
import os
from pathlib import Path
from load import get_pdf_document_paths

doc_path = Path(os.getcwd()).resolve().parents[0] / "documents" / "system"
pdf_files = get_pdf_document_paths(doc_path)
test_file = pdf_files[0]

print(doc_path)

C:\Users\90012\OneDrive\桌面\Boat_agent\agent\documents\system


C:\Users\90012\anaconda3\envs\boat_agent\Lib\site-packages\pypdf\_crypt_providers\_cryptography.py:32: CryptographyDeprecationWarning: ARC4 has been moved to cryptography.hazmat.decrepit.ciphers.algorithms.ARC4 and will be removed from cryptography.hazmat.primitives.ciphers.algorithms in 48.0.0.
  from cryptography.hazmat.primitives.ciphers.algorithms import AES, ARC4


In [2]:
'附錄3 International Convention on Standards of Training,'.endswith(',')

True

In [3]:
from pdfminer.high_level import extract_pages
from pdfminer.layout import LTTextBoxHorizontal, LTTextLineHorizontal, LTChar

def extract_bold_titles_with_comma_merge(pdf_path):
    """
    從 PDF 中提取粗體字標題，基於字體大小組合標題，並合併逗號結尾行與下一行。
    :param pdf_path: PDF 文件路徑
    :return: 包含完整標題的列表
    """
    bold_titles = []
    temp_title = ""  # 臨時存儲多行標題
    prev_font_size = None  # 上一行的字體大小
    merge_next = False  # 是否將下一行合併到當前行

    for page_layout in extract_pages(pdf_path):
        for element in page_layout:
            if isinstance(element, LTTextBoxHorizontal):
                for text_line in element:
                    if isinstance(text_line, LTTextLineHorizontal):
                        is_bold = False
                        line_text = ""
                        font_size = None  # 當前行的字體大小

                        for char in text_line:
                            if isinstance(char, LTChar):
                                # 設置當前字體大小
                                if font_size is None:
                                    font_size = round(char.size, 2)  # 四捨五入保留2位小數
                                # 判斷是否為粗體
                                if "Bold" in char.fontname:
                                    is_bold = True
                                line_text += char.get_text()

                        # 如果當前行是粗體，處理標題組合
                        if is_bold:
                            line_text = line_text.strip()  # 去除首尾空白

                            if merge_next:
                                # 合併上一行逗號結尾與當前行
                                temp_title += " " + line_text
                                merge_next = False
                            elif prev_font_size is None or font_size == prev_font_size:
                                # 字體大小相同，合併為一段
                                temp_title += " " + line_text if temp_title else line_text
                            else:
                                # 字體大小不同，存儲上一段標題
                                if temp_title:
                                    bold_titles.append(temp_title.strip())
                                temp_title = line_text

                            # 如果當前行以逗號結尾，標記與下一行合併
                            if line_text.endswith(","):
                                merge_next = True

                            prev_font_size = font_size  # 更新上一行字體大小
                        else:
                            # 遇到非粗體行，存儲已完成的標題
                            if temp_title:
                                bold_titles.append(temp_title.strip())
                                temp_title = ""
                            prev_font_size = None

    # 添加最後的臨時標題
    if temp_title:
        bold_titles.append(temp_title.strip())

    return bold_titles

# 使用示例
titles = extract_bold_titles_with_comma_merge(pdf_files[0])
print("合併換行後的粗體字標題:", titles)


Ignoring (part of) ToUnicode map because the PDF data does not conform to the format. This could result in (cid) values in the output. The start object is not a byte.


合併換行後的粗體字標題: ['國家運輸安全調查委員會 重大運輸事故調查報告', '1120603臺馬之星客貨船於連江縣福澳港外海失去動力事故', '事故簡述', '船舶損壞情況', '人員資料及配置', '載客狀況', '天氣及海象', '紀錄器相關資料', 'VDR紀錄資料', '機艙監控系統紀錄資料', 'AIS紀錄資料', '紀錄資料整合', '燃油系統與失去動力過程', '燃油系統說明', '船舶喪失動力過程', '臺馬之星恢復電力過程', '復歸按鈕', '重油日用櫃存量', '航程重油油耗', '淨油機高溫警報', '組織與管理', '臺馬之星臨時評鑑', '臺馬之星勞務合約', '臺馬之星交還船', '臺馬之星安全管理', '相關法規及文件', '船舶安全營運與防止污染管理規則', '交通部航港局「船舶安全營運與防止污染管理制度」指導原則', '航海人員訓練、發證及航行當值標準國際公約', '國際安全管理章程', '訪談紀錄 臺馬之星輪機長訪談摘要', '臺馬之星大管輪訪談摘要', '臺馬之星二管輪訪談摘要', '機電技師訪談摘要', '其他資料', '臺北時間  重', '件 資', '源 5月24日 5月25日', '6月1日', '6月2日 2210時 6月3日 0111:18時', '0416:43時 2號主機燃油低油壓警報。', '0422:27時 1號主機燃油低油壓警報。', '0423:22時 主機發生過載警報。', '0426:13時 2號發電機ACB跳脫警報。', '0428:31時 1號發電機ACB跳脫警報。', '0432:03時 3號發電機ACB跳脫，全船失去電力。', '0432:19時 1號主機與2號主機自動停俥，全船失去動力。 機艙監控系統紀錄 約1020時 東彥1號拖船拖帶臺馬之星往福澳港。', '約1500時 臺馬之星靠泊福澳港。', '分析', '臺馬之星失去動力原因', '開航前重油日用櫃存量評估', '主配電盤ACB復歸程序與操作', '事故前當值輪機員作為', '航政監理機關對船舶評鑑機制', '船舶交還船操作確認之督導機制', '結論', '與可能肇因有關之調查發現', '與風險有關之調查發現', '其它發現', '與可能肇因有關之調查發現', '與風險有關之調查發現', '其它發現', '運輸安全改善建議'

In [4]:
 titles

['國家運輸安全調查委員會 重大運輸事故調查報告',
 '1120603臺馬之星客貨船於連江縣福澳港外海失去動力事故',
 '事故簡述',
 '船舶損壞情況',
 '人員資料及配置',
 '載客狀況',
 '天氣及海象',
 '紀錄器相關資料',
 'VDR紀錄資料',
 '機艙監控系統紀錄資料',
 'AIS紀錄資料',
 '紀錄資料整合',
 '燃油系統與失去動力過程',
 '燃油系統說明',
 '船舶喪失動力過程',
 '臺馬之星恢復電力過程',
 '復歸按鈕',
 '重油日用櫃存量',
 '航程重油油耗',
 '淨油機高溫警報',
 '組織與管理',
 '臺馬之星臨時評鑑',
 '臺馬之星勞務合約',
 '臺馬之星交還船',
 '臺馬之星安全管理',
 '相關法規及文件',
 '船舶安全營運與防止污染管理規則',
 '交通部航港局「船舶安全營運與防止污染管理制度」指導原則',
 '航海人員訓練、發證及航行當值標準國際公約',
 '國際安全管理章程',
 '訪談紀錄 臺馬之星輪機長訪談摘要',
 '臺馬之星大管輪訪談摘要',
 '臺馬之星二管輪訪談摘要',
 '機電技師訪談摘要',
 '其他資料',
 '臺北時間  重',
 '件 資',
 '源 5月24日 5月25日',
 '6月1日',
 '6月2日 2210時 6月3日 0111:18時',
 '0416:43時 2號主機燃油低油壓警報。',
 '0422:27時 1號主機燃油低油壓警報。',
 '0423:22時 主機發生過載警報。',
 '0426:13時 2號發電機ACB跳脫警報。',
 '0428:31時 1號發電機ACB跳脫警報。',
 '0432:03時 3號發電機ACB跳脫，全船失去電力。',
 '0432:19時 1號主機與2號主機自動停俥，全船失去動力。 機艙監控系統紀錄 約1020時 東彥1號拖船拖帶臺馬之星往福澳港。',
 '約1500時 臺馬之星靠泊福澳港。',
 '分析',
 '臺馬之星失去動力原因',
 '開航前重油日用櫃存量評估',
 '主配電盤ACB復歸程序與操作',
 '事故前當值輪機員作為',
 '航政監理機關對船舶評鑑機制',
 '船舶交還船操作確認之督導機制',
 '結論',
 '與可能肇因有關之調查發現',
 '與風險有關之調查發現',
 '其它發現',
 '與

In [5]:
def merge_title(bold_titles):
    index = 0
        
    while index < len(bold_titles) - 1:
        if bold_titles[index].endswith(","):
            # 合併當前標題與下一標題，並取代當前標題
            bold_titles[index] += " " + bold_titles[index + 1]
            del bold_titles[index + 1]  # 刪除已合併的下一標題
        else:
            index += 1
    
    return bold_titles

merge_title(titles)

['國家運輸安全調查委員會 重大運輸事故調查報告',
 '1120603臺馬之星客貨船於連江縣福澳港外海失去動力事故',
 '事故簡述',
 '船舶損壞情況',
 '人員資料及配置',
 '載客狀況',
 '天氣及海象',
 '紀錄器相關資料',
 'VDR紀錄資料',
 '機艙監控系統紀錄資料',
 'AIS紀錄資料',
 '紀錄資料整合',
 '燃油系統與失去動力過程',
 '燃油系統說明',
 '船舶喪失動力過程',
 '臺馬之星恢復電力過程',
 '復歸按鈕',
 '重油日用櫃存量',
 '航程重油油耗',
 '淨油機高溫警報',
 '組織與管理',
 '臺馬之星臨時評鑑',
 '臺馬之星勞務合約',
 '臺馬之星交還船',
 '臺馬之星安全管理',
 '相關法規及文件',
 '船舶安全營運與防止污染管理規則',
 '交通部航港局「船舶安全營運與防止污染管理制度」指導原則',
 '航海人員訓練、發證及航行當值標準國際公約',
 '國際安全管理章程',
 '訪談紀錄 臺馬之星輪機長訪談摘要',
 '臺馬之星大管輪訪談摘要',
 '臺馬之星二管輪訪談摘要',
 '機電技師訪談摘要',
 '其他資料',
 '臺北時間  重',
 '件 資',
 '源 5月24日 5月25日',
 '6月1日',
 '6月2日 2210時 6月3日 0111:18時',
 '0416:43時 2號主機燃油低油壓警報。',
 '0422:27時 1號主機燃油低油壓警報。',
 '0423:22時 主機發生過載警報。',
 '0426:13時 2號發電機ACB跳脫警報。',
 '0428:31時 1號發電機ACB跳脫警報。',
 '0432:03時 3號發電機ACB跳脫，全船失去電力。',
 '0432:19時 1號主機與2號主機自動停俥，全船失去動力。 機艙監控系統紀錄 約1020時 東彥1號拖船拖帶臺馬之星往福澳港。',
 '約1500時 臺馬之星靠泊福澳港。',
 '分析',
 '臺馬之星失去動力原因',
 '開航前重油日用櫃存量評估',
 '主配電盤ACB復歸程序與操作',
 '事故前當值輪機員作為',
 '航政監理機關對船舶評鑑機制',
 '船舶交還船操作確認之督導機制',
 '結論',
 '與可能肇因有關之調查發現',
 '與風險有關之調查發現',
 '其它發現',
 '與

In [6]:
import pdfplumber
import pandas as pd

def extract_table_with_pdfplumber_to_list(pdf_path):
    all_tables = []  # 存儲所有頁面的表格

    with pdfplumber.open(pdf_path) as pdf:
        for i, page in enumerate(pdf.pages):
            print(f"正在處理第 {i+1} 頁...")
            table = page.extract_table()  # 提取表格
            if table:
                # 過濾掉表格中的 None 值
                filtered_table = [[cell if cell is not None else "" for cell in row] for row in table]

                # 將過濾後的表格數據加入列表
                all_tables.append(filtered_table)

                # 顯示處理後的表格
                print(f"第 {i+1} 頁表格（過濾後）:\n{filtered_table}\n")

    return all_tables  # 返回純淨的表格列表

# 使用示例
pdf_path = pdf_files[0]  # 替換為你的 PDF 文件
tables = extract_table_with_pdfplumber_to_list(pdf_path)

# 打印最終提取的表格數據
print("提取的表格數據（純列表格式）:")
for idx, table in enumerate(tables, 1):
    print(f"表格 {idx}:\n", table)

Ignoring (part of) ToUnicode map because the PDF data does not conform to the format. This could result in (cid) values in the output. The start object is not a byte.


正在處理第 1 頁...
第 1 頁表格（過濾後）:
[['', '發布日期：', '', '', '民國 113 年 7 月 15 日', '']]

正在處理第 2 頁...
正在處理第 3 頁...
正在處理第 4 頁...
正在處理第 5 頁...
正在處理第 6 頁...
正在處理第 7 頁...
正在處理第 8 頁...
正在處理第 9 頁...
正在處理第 10 頁...
正在處理第 11 頁...
正在處理第 12 頁...
正在處理第 13 頁...
正在處理第 14 頁...
正在處理第 15 頁...
正在處理第 16 頁...
正在處理第 17 頁...
正在處理第 18 頁...
正在處理第 19 頁...
正在處理第 20 頁...
正在處理第 21 頁...
正在處理第 22 頁...
正在處理第 23 頁...
正在處理第 24 頁...
正在處理第 25 頁...
正在處理第 26 頁...
正在處理第 27 頁...
正在處理第 28 頁...
正在處理第 29 頁...
正在處理第 30 頁...
正在處理第 31 頁...
正在處理第 32 頁...
第 32 頁表格（過濾後）:
[['', '臺北時間', '', '', '重 要 事 件', '', '', '資 料 來 源', ''], ['5月 24日', '', '', '世洋公司接受航港局臨時評鑑，並取得臨時符\n合證書。', '', '', '臺馬之星安全管理文件', '', ''], ['5月 25日', '', '', '世洋公司與佑啟新公司登輪臺馬之星，並與新\n華航業交接。', '', '', '組織與管理文件', '', ''], ['6月 1日', '', '', '臺馬之星正式交接由世洋公司管理，並取得航\n港局核發之臨時船舶安全管理證書。', '', '', '組織與管理/\n臺馬之星安全管理文件', '', ''], ['6月 2日\n2210時', '', '', '臺馬之星自基隆港啟航，預計前往福澳港；開\n航前輪機長推估重油日用櫃存量足夠航行至\n福澳港。', '', '', '事故簡述/\n輪機長訪談摘要', '', ''], ['', '6月 3日', '', '重油日用櫃低位警報作動，液位為 39.8%。', '', '', '機艙監

In [7]:
len(tables)

3

In [8]:
tables

[[['', '發布日期：', '', '', '民國 113 年 7 月 15 日', '']],
 [['', '臺北時間', '', '', '重 要 事 件', '', '', '資 料 來 源', ''],
  ['5月 24日',
   '',
   '',
   '世洋公司接受航港局臨時評鑑，並取得臨時符\n合證書。',
   '',
   '',
   '臺馬之星安全管理文件',
   '',
   ''],
  ['5月 25日',
   '',
   '',
   '世洋公司與佑啟新公司登輪臺馬之星，並與新\n華航業交接。',
   '',
   '',
   '組織與管理文件',
   '',
   ''],
  ['6月 1日',
   '',
   '',
   '臺馬之星正式交接由世洋公司管理，並取得航\n港局核發之臨時船舶安全管理證書。',
   '',
   '',
   '組織與管理/\n臺馬之星安全管理文件',
   '',
   ''],
  ['6月 2日\n2210時',
   '',
   '',
   '臺馬之星自基隆港啟航，預計前往福澳港；開\n航前輪機長推估重油日用櫃存量足夠航行至\n福澳港。',
   '',
   '',
   '事故簡述/\n輪機長訪談摘要',
   '',
   ''],
  ['', '6月 3日', '', '重油日用櫃低位警報作動，液位為 39.8%。', '', '', '機艙監控系統紀錄', '', ''],
  ['', '0111:18時', '', '', '', '', '', '', ''],
  ['0416:43時', '', '', '2號主機燃油低油壓警報。', '', '', '機艙監控系統紀錄', '', ''],
  ['0422:27時', '', '', '1號主機燃油低油壓警報。', '', '', '機艙監控系統紀錄', '', ''],
  ['0423:22時', '', '', '主機發生過載警報。', '', '', '機艙監控系統紀錄', '', ''],
  ['0426:13時', '', '', '2號發電機 ACB跳脫警報。', '', '', '機艙監控系統紀錄', '', ''],
  ['0428:31時', '', '', '

In [9]:
 titles

['國家運輸安全調查委員會 重大運輸事故調查報告',
 '1120603臺馬之星客貨船於連江縣福澳港外海失去動力事故',
 '事故簡述',
 '船舶損壞情況',
 '人員資料及配置',
 '載客狀況',
 '天氣及海象',
 '紀錄器相關資料',
 'VDR紀錄資料',
 '機艙監控系統紀錄資料',
 'AIS紀錄資料',
 '紀錄資料整合',
 '燃油系統與失去動力過程',
 '燃油系統說明',
 '船舶喪失動力過程',
 '臺馬之星恢復電力過程',
 '復歸按鈕',
 '重油日用櫃存量',
 '航程重油油耗',
 '淨油機高溫警報',
 '組織與管理',
 '臺馬之星臨時評鑑',
 '臺馬之星勞務合約',
 '臺馬之星交還船',
 '臺馬之星安全管理',
 '相關法規及文件',
 '船舶安全營運與防止污染管理規則',
 '交通部航港局「船舶安全營運與防止污染管理制度」指導原則',
 '航海人員訓練、發證及航行當值標準國際公約',
 '國際安全管理章程',
 '訪談紀錄 臺馬之星輪機長訪談摘要',
 '臺馬之星大管輪訪談摘要',
 '臺馬之星二管輪訪談摘要',
 '機電技師訪談摘要',
 '其他資料',
 '臺北時間  重',
 '件 資',
 '源 5月24日 5月25日',
 '6月1日',
 '6月2日 2210時 6月3日 0111:18時',
 '0416:43時 2號主機燃油低油壓警報。',
 '0422:27時 1號主機燃油低油壓警報。',
 '0423:22時 主機發生過載警報。',
 '0426:13時 2號發電機ACB跳脫警報。',
 '0428:31時 1號發電機ACB跳脫警報。',
 '0432:03時 3號發電機ACB跳脫，全船失去電力。',
 '0432:19時 1號主機與2號主機自動停俥，全船失去動力。 機艙監控系統紀錄 約1020時 東彥1號拖船拖帶臺馬之星往福澳港。',
 '約1500時 臺馬之星靠泊福澳港。',
 '分析',
 '臺馬之星失去動力原因',
 '開航前重油日用櫃存量評估',
 '主配電盤ACB復歸程序與操作',
 '事故前當值輪機員作為',
 '航政監理機關對船舶評鑑機制',
 '船舶交還船操作確認之督導機制',
 '結論',
 '與可能肇因有關之調查發現',
 '與風險有關之調查發現',
 '其它發現',
 '與

In [10]:
def clean_titles(titles, tables):
    """
    從 titles 中移除出現在 tables 中的內容。
    tables 的文字會去除空格並合併，作為過濾基準。
    titles 中的內容如果包含表格的任何文字（子集合檢查），則移除。

    :param titles: 原始粗體標題列表 (1D list)
    :param tables: 表格數據 (嵌套列表形式的表格)
    :return: 過濾後的粗體標題列表
    """

    def normalize_table_text(text):
        """
        將表格文字中的空格移除並合併。
        :param text: 表格中的文字
        :return: 合併後的文字
        """
        return text.replace(" ", "")  # 移除所有空格

    # 將表格內容合併處理，生成過濾基準集合
    table_texts = set(
        normalize_table_text(cell)
        for table in tables
        for row in table
        for cell in row if cell
    )

    # 過濾 titles 中出現在表格內容的部分
    filtered_titles = []
    for title in titles:
        # 移除空格，並檢查是否包含表格文字
        if not any(table_text in title.replace(" ", "") for table_text in table_texts):
            filtered_titles.append(title)

    return filtered_titles
# 過濾後的結果
filtered_titles = clean_titles(titles, tables)
print("過濾後的粗體標題:", filtered_titles)

過濾後的粗體標題: ['國家運輸安全調查委員會 重大運輸事故調查報告', '事故簡述', '船舶損壞情況', '人員資料及配置', '載客狀況', '天氣及海象', '紀錄器相關資料', 'VDR紀錄資料', '燃油系統與失去動力過程', '燃油系統說明', '船舶喪失動力過程', '臺馬之星恢復電力過程', '復歸按鈕', '重油日用櫃存量', '航程重油油耗', '淨油機高溫警報', '組織與管理', '臺馬之星臨時評鑑', '臺馬之星勞務合約', '臺馬之星交還船', '臺馬之星安全管理', '相關法規及文件', '船舶安全營運與防止污染管理規則', '航海人員訓練、發證及航行當值標準國際公約', '國際安全管理章程', '訪談紀錄 臺馬之星輪機長訪談摘要', '臺馬之星大管輪訪談摘要', '臺馬之星二管輪訪談摘要', '機電技師訪談摘要', '其他資料', '件 資', '分析', '臺馬之星失去動力原因', '開航前重油日用櫃存量評估', '主配電盤ACB復歸程序與操作', '事故前當值輪機員作為', '航政監理機關對船舶評鑑機制', '船舶交還船操作確認之督導機制', '結論', '與可能肇因有關之調查發現', '與風險有關之調查發現', '其它發現', '與可能肇因有關之調查發現', '與風險有關之調查發現', '其它發現', '運輸安全改善建議', '連江縣馬祖連江航業有限公司', '表', '船舶資料', '號 途 質 位 長 寬 深 司 人 期 點 式 商 構', '附錄1臺馬之星交船證明', '附錄2 輪機部離港準備查核表', '附錄3 International Convention on Standards of Training, Certification and Watchkeeping for Seafarers, STCW', '附錄4 International Safety Management Code, ISM Code', 'PART A 1 GENERAL', '6 RESOURCE AND PERSONMEL', '10 MAINTENANCE OF THE SHIP AND EQUIPMENT', 'PART B 14 INTERIM CERTIFICATION']


In [11]:
filtered_titles

['國家運輸安全調查委員會 重大運輸事故調查報告',
 '事故簡述',
 '船舶損壞情況',
 '人員資料及配置',
 '載客狀況',
 '天氣及海象',
 '紀錄器相關資料',
 'VDR紀錄資料',
 '燃油系統與失去動力過程',
 '燃油系統說明',
 '船舶喪失動力過程',
 '臺馬之星恢復電力過程',
 '復歸按鈕',
 '重油日用櫃存量',
 '航程重油油耗',
 '淨油機高溫警報',
 '組織與管理',
 '臺馬之星臨時評鑑',
 '臺馬之星勞務合約',
 '臺馬之星交還船',
 '臺馬之星安全管理',
 '相關法規及文件',
 '船舶安全營運與防止污染管理規則',
 '航海人員訓練、發證及航行當值標準國際公約',
 '國際安全管理章程',
 '訪談紀錄 臺馬之星輪機長訪談摘要',
 '臺馬之星大管輪訪談摘要',
 '臺馬之星二管輪訪談摘要',
 '機電技師訪談摘要',
 '其他資料',
 '件 資',
 '分析',
 '臺馬之星失去動力原因',
 '開航前重油日用櫃存量評估',
 '主配電盤ACB復歸程序與操作',
 '事故前當值輪機員作為',
 '航政監理機關對船舶評鑑機制',
 '船舶交還船操作確認之督導機制',
 '結論',
 '與可能肇因有關之調查發現',
 '與風險有關之調查發現',
 '其它發現',
 '與可能肇因有關之調查發現',
 '與風險有關之調查發現',
 '其它發現',
 '運輸安全改善建議',
 '連江縣馬祖連江航業有限公司',
 '表',
 '船舶資料',
 '號 途 質 位 長 寬 深 司 人 期 點 式 商 構',
 '附錄1臺馬之星交船證明',
 '附錄2 輪機部離港準備查核表',
 '附錄3 International Convention on Standards of Training, Certification and Watchkeeping for Seafarers, STCW',
 '附錄4 International Safety Management Code, ISM Code',
 'PART A 1 GENERAL',
 '6 RESOURCE AND PERSONMEL',
 '10 MAINTENANCE OF THE SHIP AND EQUIPMENT',
 'PART B 14 I